# Astelias - Accumulations



In [1]:
import sys
sys.path.insert(0, '../../../lib/')

import tabula
import numpy as np
import pandas as pd
import importlib

import pdfexport
importlib.reload(pdfexport)

from pdfexport import *

In [ ]:
# Read pdf into DataFrame
df = tabula.read_pdf("2017+(ENGLISH)+Disclosure+Code+Report_SWITZERLAND_12062018_V1.0.pdf", pages='all', lattice=True)

In [ ]:
df_export = df.copy()

#Shift all where column0 is empty
df_export[df_export[df_export.columns[0]].notna()] = df_export[df_export[df_export.columns[0]].notna()].shift(1, axis='columns')

#Drop Columns
df_export = df_export.drop(df.columns[0],axis=1)
df_export = df_export.drop(df.columns[9:],axis=1)

#Rename
df_export.columns = fix_columns_accumulations[0: 8]

#fill na
df_export = df_export.fillna("")

# Select
amount = df_export[df_export['type'].str.contains('Aggregate amount') & df_export['total'] > 0]
amount = amount[amount.duplicated('total')]
recipients = df_export[df_export['type'].str.contains('Number of Recipients') & df_export['total'] > 0]
recipients = recipients[recipients.duplicated('total')]
rnd = df_export[df_export.type.str.contains('Transfers of Value re Research')& df_export['donations_grants'] > 0]
rnd = rnd[rnd.duplicated('donations_grants')]

#concat
df_export = pd.concat([amount, recipients, rnd])

#Numberize 
df_export = df_export.replace('N/V', np.NaN)
df_export = cleanup_number(df_export)
df_export = amounts_to_number(df_export)

df_export = df_export.reset_index(drop=True)
df_export.iloc[0,0] = "hcp_amount"
df_export.iloc[1,0] = "hco_amount"
df_export.iloc[2,0] = "hcp_count"
df_export.iloc[3,0] = "hco_count"
df_export.iloc[4,0] = "rnd"

#Fix RnD
df_export.iloc[4,7] = df_export.iloc[4,1]
df_export.iloc[4,1] = np.NaN

export_acumulations(df_export, 'astellas')